In [ ]:
import pandas as pd
import numpy as np

In [ ]:
### The Reset Code ###

# Read Data
df = pd.read_excel("01_School-data_raw.xlsx")

# Let's remove some irrelevant columns.
df_dropped = df.drop(['Board Number','School Number','Building Suite','P.O. Box','Street','Phone Number','Fax Number', 'Province', 'Municipality', 'School Website','Board Website',
         'Percentage of Students Whose First Language Is Not French', 'Percentage of Students Who Are New to Canada from a Non-French Speaking Country',
         'Extract Date'],
        axis = 1)

# Filtering only for Toronto
Toronto_df = df_dropped.loc[(df_dropped['School Language'] == 'English') & (df_dropped['Postal Code'].str.startswith('M', na = False))]

In [ ]:
### Data Transformation ###

# Create list of columns for fields starting with "Percentage"
percentage_cols = Toronto_df[Toronto_df.filter(regex = 'Percentage').columns].columns.tolist()

# Create a new list of columns, replacing Percentage with Number
number_cols = [i.replace("Percentage","Number") for i in percentage_cols]

# Transform the string percentage fields into numeric values
Toronto_df[percentage_cols] = Toronto_df[percentage_cols].replace(r'%','',regex = True)
Toronto_df[percentage_cols] = (Toronto_df[percentage_cols].apply(pd.to_numeric, errors = 'coerce'))/100

# Remove Enrolment NA fields, turn field to numeric, remove new NA schools coerced from suppressed data
Toronto_df = Toronto_df[Toronto_df['Enrolment'].notna()]
Toronto_df['Enrolment'] = Toronto_df['Enrolment'].apply(pd.to_numeric, errors = 'coerce')
Toronto_df = Toronto_df[Toronto_df['Enrolment'].notna()]

# Creating Number columns from percentage columns
for i, j in zip(percentage_cols, number_cols):
    Toronto_df[j] = round(Toronto_df['Enrolment'] * Toronto_df[i])

# Create column list of fields starting with "Change"
change_cols = Toronto_df[Toronto_df.filter(regex = 'Change').columns].columns.tolist()

# Turning Change columns into calculatable fields
Toronto_df[change_cols] = (Toronto_df[change_cols]/100) + 1

# Column list of Number of 2016 students
number_2016cols = ['2016 Grade 3 Reading',
 '2016 Grade 3 Writing',
 '2016 Grade 3 Math',
 '2016 Grade 6 Reading',
 '2016 Grade 6 Writing',
 '2016 Grade 6 Math',
 '2016 Grade 9 Academic Math',
 '2016 Grade 9 Applied Math',
 '2016 Grade 10 OSSLT']

# Getting a column list of only 2019 Numbers
number_2019cols = ['Number of Grade 3 Students Achieving the Provincial Standard in Reading',
 'Number of Grade 3 Students Achieving the Provincial Standard in Writing',
 'Number of Grade 3 Students Achieving the Provincial Standard in Mathematics',
 'Number of Grade 6 Students Achieving the Provincial Standard in Reading',
 'Number of Grade 6 Students Achieving the Provincial Standard in Writing',
 'Number of Grade 6 Students Achieving the Provincial Standard in Mathematics',
 'Number of Grade 9 Students Achieving the Provincial Standard in Academic Mathematics',
 'Number of Grade 9 Students Achieving the Provincial Standard in Applied Mathematics',
 'Number of Students That Passed the Grade 10 OSSLT on Their First Attempt']

# Creating 2016 Number columns from 
for i, j, k in zip(number_2019cols, change_cols, number_2016cols):
  Toronto_df[k] = round(Toronto_df[i]/Toronto_df[j])

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
### Clean up old columns ###
Toronto_df_cleaned = Toronto_df.drop(percentage_cols, axis = 1)
Toronto_df_cleaned = Toronto_df_cleaned.drop(change_cols, axis = 1)

In [ ]:
### Renaming and rearranging columns ###

# Renaming 2019 columns. First create a list of the new names in order
number_2019cols_new = ['2019 Grade 3 Reading',
 '2019 Grade 3 Writing',
 '2019 Grade 3 Math',
 '2019 Grade 6 Reading',
 '2019 Grade 6 Writing',
 '2019 Grade 6 Math',
 '2019 Grade 9 Academic Math',
 '2019 Grade 9 Applied Math',
 '2019 Grade 10 OSSLT']

# Zip the list of new names with old names into a dictionary
cols2019_dict = dict(zip(number_2019cols, number_2019cols_new))

# The renaming process
Toronto_df_cleaned = Toronto_df_cleaned.rename(columns=cols2019_dict)

# Reordering columns. First create a list of column names
cleaned_cols_list = Toronto_df_cleaned.columns.tolist()

# Using the information obtained above, create a new list with the columns you want them to be
new_cleaned_cols_list =  ['Board Name',
 'Board Type',
 'School Name',
 'School Type',
 'School Special Condition Code',
 'School Level',
 'School Language',
 'Grade Range',
 'City',
 'Postal Code',
 'Enrolment',
 'Latitude',
 'Longitude',
 'Number of Students Whose First Language Is Not English',
 'Number of Students Who Are New to Canada from a Non-English Speaking Country',
 'Number of Students Receiving Special Education Services',
 'Number of Students Identified as Gifted',
 'Number of School-Aged Children Who Live in Low-Income Households',
 'Number of Students Whose Parents Have Some University Education',
 '2016 Grade 3 Reading',
 '2019 Grade 3 Reading',
 '2016 Grade 3 Writing',
 '2019 Grade 3 Writing',
 '2016 Grade 3 Math',
 '2019 Grade 3 Math',
 '2016 Grade 6 Reading',
 '2019 Grade 6 Reading',
 '2016 Grade 6 Writing',
 '2019 Grade 6 Writing',
 '2016 Grade 6 Math',
 '2019 Grade 6 Math',
 '2016 Grade 9 Academic Math',
 '2019 Grade 9 Academic Math',
 '2016 Grade 9 Applied Math',
 '2019 Grade 9 Applied Math',
 '2016 Grade 10 OSSLT',
 '2019 Grade 10 OSSLT']

 # Reading the data in the new order with the new list
Toronto_df_cleaned = Toronto_df_cleaned[new_cleaned_cols_list]

 # Manual renaming for the final columns
Toronto_df_cleaned = Toronto_df_cleaned.rename(columns={'Number of Students Whose First Language Is Not English':'First Language Not English',
                           'Number of Students Who Are New to Canada from a Non-English Speaking Country':'New to Canada from Non-English Speaking Country',
                           'Number of Students Receiving Special Education Services':'Receiving Special Education Services',
                           'Number of Students Identified as Gifted':'Identified as Gifted',
                           'Number of School-Aged Children Who Live in Low-Income Households':'Live in Low-Income Households',
                           'Number of Students Whose Parents Have Some University Education':'Parents Have Some University Education'})

Toronto_df_cleaned['FSA'] = Toronto_df_cleaned['Postal Code'].str[:3]

So what am I trying to do? What are the possible questions?
- I want to look at location of the school, the type of the school, %of students that relates to English fields, gifted, all the % stats for grades, income.
- Are there more "gifted" students in low-income schools (Can that be answered?)
- English and Math, which subject are students better in?
- How common are students whose first language is not English in low-income neighbourhoods

To Do:
- Filter location down to Toronto, so take the first 3 characters of the postal code, filter those that start with M
- Filter schools down to English only
- Delete columns Building Site, P.O. Box, 


In [ ]:
# Issue 1: The Grade Range is split between 9-12 and a bunch of other ranges. Which means all the Grade 9-12 schools will not have data for Grade 3 and 6, and vice versa.
#          Hence I will 
secondary_cols = Toronto_df_cleaned[Toronto_df_cleaned.filter(regex = 'Grade 9').columns].columns.tolist() + Toronto_df_cleaned[Toronto_df_cleaned.filter(regex = 'Grade 10').columns].columns.tolist()
Toronto_df_young = Toronto_df_cleaned.drop(secondary_cols,
                              axis = 1)
Toronto_df_young = Toronto_df_young.loc[df['Grade Range'] != "9-12"]
Toronto_df_young.fillna("N/A", inplace=True)

In [ ]:
Toronto_df_secondary = Toronto_df_cleaned.loc[df['Grade Range'] == "9-12"]
elementary_cols = Toronto_df_cleaned[Toronto_df_cleaned.filter(regex = 'Grade 3').columns].columns.tolist() + Toronto_df_cleaned[Toronto_df_cleaned.filter(regex = 'Grade 6').columns].columns.tolist()
Toronto_df_secondary = Toronto_df_secondary.drop(elementary_cols,
                                     axis = 1)
Toronto_df_secondary.fillna("N/A", inplace=True)

In [ ]:
Toronto_df_young.to_csv("02_Toronto-elemid_cleaned.csv", index=False)
Toronto_df_secondary.to_csv("03_Toronto-seconday_cleaned.csv", index=False)